In [2]:
import cv2
import numpy as np


In [5]:
# ==== 1. Load Images ====

num_imgs = int(input("How many images do you want to stitch? (min 2): "))

paths = []
for i in range(num_imgs):
    p = input(f"Enter path for image {i+1}: ")
    paths.append(p)

images = []
for i, p in enumerate(paths):
    img = cv2.imread(p)

    if img is None:
        print(f"[ERROR] Could not load: {p}")
        continue

    images.append(img)
    print(f"\n[INFO] Loaded Image {i+1}: {p}")

    # Show image correctly (VS Code friendly)
    cv2.imshow(f"Image {i+1}", img)
    cv2.waitKey(1000)        # show for 1 second (you can change this)
    cv2.destroyAllWindows()

if len(images) < 2:
    raise ValueError("Need at least 2 valid images to stitch!")



[INFO] Loaded Image 1: C:\Users\daksh\MachineLearning\OpenCV\WhatsApp Image 2025-11-18 at 11.12.46 PM.jpeg

[INFO] Loaded Image 2: C:\Users\daksh\MachineLearning\OpenCV\WhatsApp Image 2025-11-18 at 11.12.47 PM (1).jpeg

[INFO] Loaded Image 3: C:\Users\daksh\MachineLearning\OpenCV\WhatsApp Image 2025-11-18 at 11.12.48 PM.jpeg


In [8]:
gray_images = []

for i, img in enumerate(images):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_images.append(gray)
    print(f"[INFO] Converted Image {i+1} to grayscale")

# Show grayscale images (VS Code compatible)
print("\n[INFO] Showing grayscale versions:")
for i, g in enumerate(gray_images):
    print(f"Gray Image {i+1}")

    cv2.imshow(f"Grayscale Image {i+1}", g)
    cv2.waitKey(1000)   # show for 1 second (you can change it)
    cv2.destroyAllWindows()


[INFO] Converted Image 1 to grayscale
[INFO] Converted Image 2 to grayscale
[INFO] Converted Image 3 to grayscale

[INFO] Showing grayscale versions:
Gray Image 1
Gray Image 2
Gray Image 3


In [9]:
orb = cv2.ORB_create(nfeatures=4000)

kp_list = []
des_list = []

for i, gray in enumerate(gray_images):

    # Detect keypoints + descriptors
    keypts, des = orb.detectAndCompute(gray, None)
    print(f"[INFO] Image {i+1}: {len(keypts)} keypoints detected")

    kp_list.append(keypts)
    des_list.append(des)

    # Visualize keypoints
    kp_img = cv2.drawKeypoints(
        gray, keypts, None,
        color=(0, 255, 0),
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    )

    # Show in VS Code
    cv2.imshow(f"Keypoints Image {i+1}", kp_img)
    cv2.waitKey(1000)     # show for 1 second (change to 0 for manual close)
    cv2.destroyAllWindows()


[INFO] Image 1: 2003 keypoints detected
[INFO] Image 2: 2812 keypoints detected
[INFO] Image 3: 4000 keypoints detected


In [13]:
def match_features_orb(desc1, desc2):
    # safety checks
    if desc1 is None or desc2 is None:
        print("[ERROR] Descriptor missing — cannot match")
        return []

    if len(desc1) == 0 or len(desc2) == 0:
        print("[ERROR] Empty descriptor set — cannot match")
        return []

    # ORB uses Hamming distance
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    # KNN matches
    matches = bf.knnMatch(desc1, desc2, k=2)

    # Lowe’s ratio test
    good = [m for m, n in matches if m.distance < 0.75 * n.distance]

    print(f"[INFO] Good Matches: {len(good)}")
    return good



# ============================================================
# AUTOMATIC MATCHING FOR N IMAGES (VS Code)
# ============================================================

all_matches = []

for i in range(len(des_list) - 1):
    print(f"\n=== Matching Image {i+1} with Image {i+2} ===")

    desc1 = des_list[i]
    desc2 = des_list[i + 1]

    good = match_features_orb(desc1, desc2)

    all_matches.append(good)





=== Matching Image 1 with Image 2 ===
[INFO] Good Matches: 293

=== Matching Image 2 with Image 3 ===
[INFO] Good Matches: 151


In [15]:
def draw_matches(img1, kp1, img2, kp2, matches, max_show=50):
    if len(matches) == 0:
        print("[WARN] No matches to visualize")
        return

    matched_img = cv2.drawMatches(
        img1, kp1,
        img2, kp2,
        matches[:max_show], None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    cv2.imshow("Matches", matched_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [18]:
for i in range(len(all_matches)):
    print(f"\nShowing matches for Image {i+1} & Image {i+2}")
    draw_matches(images[i], kp_list[i], images[i+1], kp_list[i+1], all_matches[i])



Showing matches for Image 1 & Image 2

Showing matches for Image 2 & Image 3


In [21]:
# ==========================================================
# 4. FEATHER BLENDING FUNCTION (must come BEFORE warp_and_merge)
# ==========================================================

def feather_blend(base, overlay):
    base_gray = cv2.cvtColor(base, cv2.COLOR_BGR2GRAY)
    over_gray = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)

    base_mask = (base_gray > 0).astype(np.uint8)
    over_mask = (over_gray > 0).astype(np.uint8)

    if base_mask.max() == 0:
        return overlay
    if over_mask.max() == 0:
        return base

    dist_base = cv2.distanceTransform(base_mask, cv2.DIST_L2, 3)
    dist_over = cv2.distanceTransform(over_mask, cv2.DIST_L2, 3)

    weight_base = dist_base / (dist_base + dist_over + 1e-8)
    weight_over = dist_over / (dist_base + dist_over + 1e-8)

    weight_base = weight_base[..., None]
    weight_over = weight_over[..., None]

    blended = base * weight_base + overlay * weight_over
    return blended.astype(np.uint8)


In [25]:
# ==========================================================
# AUTO-CROP BLACK BORDERS (must come BEFORE stitching loop)
# ==========================================================

def auto_crop_black(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # create binary mask of non-black region
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    coords = cv2.findNonZero(thresh)
    if coords is None:
        return image

    x, y, w, h = cv2.boundingRect(coords)
    return image[y:y+h, x:x+w]


In [26]:
def warp_and_merge(img1, img2, H):
    # Ensure H is float64
    H = H.astype(np.float64)

    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Corners of image 2
    corners_img2 = np.float32([
        [0, 0],
        [0, h2],
        [w2, h2],
        [w2, 0]
    ]).reshape(-1, 1, 2)

    warped_corners_img2 = cv2.perspectiveTransform(corners_img2, H)

    # Corners of image 1
    corners_img1 = np.float32([
        [0, 0],
        [0, h1],
        [w1, h1],
        [w1, 0]
    ]).reshape(-1, 1, 2)

    # All corners combined
    all_corners = np.concatenate((corners_img1, warped_corners_img2), axis=0)

    # Bounding box for panorama
    xmin, ymin = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    xmax, ymax = np.int32(all_corners.max(axis=0).ravel() + 0.5)

    # Translate so image fits whole canvas
    tx, ty = -xmin, -ymin
    T = np.array([[1, 0, tx],
                  [0, 1, ty],
                  [0, 0, 1]], dtype=np.float64)

    size = (xmax - xmin, ymax - ymin)

    # Warp image 2 into the panorama frame
    warped_img2 = cv2.warpPerspective(img2, T @ H, size)

    # Place image 1 onto canvas
    canvas1 = np.zeros_like(warped_img2)
    canvas1[ty:ty + h1, tx:tx + w1] = img1

    # Blend both images
    blended = feather_blend(canvas1, warped_img2)

    return blended


In [ ]:
# ==========================================================
# 7. STITCHING LOOP (AUTOMATIC FOR N IMAGES)
# ==========================================================

panorama = images[0]   # start with first image

for i in range(1, len(images)):
    print(f"\n[STEP] Stitching image {i+1}")

    # ORB features for the current panorama
    gray_pano = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
    kp_pano, des_pano = orb.detectAndCompute(gray_pano, None)

    # Precomputed features of next image
    kp_next = kp_list[i]
    des_next = des_list[i]

    # Feature match
    matches = match_features_orb(des_pano, des_next)

    if len(matches) < 4:
        print("[ERROR] Not enough matches — skipping image", i+1)
        continue

    # Homography
    src_pts = np.float32([kp_pano[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    dst_pts = np.float32([kp_next[m.trainIdx].pt for m in matches]).reshape(-1,1,2)

    H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
    print("[INFO] Homography:")
    print(H)

    # Warp + blend
    panorama = warp_and_merge(panorama, images[i], H)

    # Crop black borders
    panorama = auto_crop_black(panorama)

    # Show progress
    cv2.imshow("Panorama Progress", panorama)
    cv2.waitKey(1)



[STEP] Stitching image 2
[INFO] Good Matches: 293
[INFO] Homography:
[[ 6.26998098e-01 -1.26874194e-02  3.45941364e+02]
 [-2.71100531e-01  7.91645587e-01  1.68171374e+02]
 [-3.75853289e-04 -1.04408919e-04  1.00000000e+00]]

[STEP] Stitching image 3
[INFO] Good Matches: 94
[INFO] Homography:
[[ 1.17888635e-01 -2.14789113e-02  9.21775654e+02]
 [-7.58742697e-01  8.87670546e-01  1.90383566e+02]
 [-9.51441653e-04 -1.50943207e-04  1.00000000e+00]]


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 102873895137 bytes in function 'cv::OutOfMemoryError'


: 